In [1]:
import pandas as pd
import os
import sys
module_path = os.path.abspath(os.path.join('/Users/balbi/Downloads/pyMorfologik-master'))
if module_path not in sys.path:
    sys.path.append(module_path)
from pymorfologik import Morfologik
from pymorfologik.parsing import ListParser
from pymorfologik.parsing import BaseParser
from pymorfologik.parsing import DictParser

import numpy as np
from __future__ import print_function
import string
from multiprocessing import cpu_count, Pool
import dask.dataframe as dd
from dask.multiprocessing import get
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def getEmotionStatusOfText(result):
    if(result < 0):
        return 'negative'
    elif(result > 0):
        return 'positive'
    else:
        return 'neutral'

In [3]:
def lemmatisation_dictParser(text):
    parser = DictParser()
    stemmer = Morfologik()
    stemming = stemmer.stem([text], parser)
    words_list = list()
    for key, val in stemming.items():
        words_list.append(val[0].encode('utf8'))
    return words_list

In [4]:
emotions = pd.read_csv('emotionsTable-with-neutralEmotion.csv')
def detectingEmotionsInText(words_list):
    numberOfWords = len(words_list)
    numberOfWordsFound = 0
    result = 0
    emotions_list = list()
    if(numberOfWords > 0):
        emotionsCounter = 0
        for el in words_list: 
            if((emotions['word'] == el)).any():
                emotions_list.append(el)
                emotionsCounter += float(emotions.loc[emotions['word'] == el]['sentiment'].mean())
                numberOfWordsFound = numberOfWordsFound + 1
        result = emotionsCounter/numberOfWords
    return [result, numberOfWordsFound, emotions_list]

In [ ]:
%%time
import sys
import re
reload(sys)
sys.setdefaultencoding('utf-8')

result = pd.read_csv('restaurant_reviews.csv')
result = result.reset_index()

def calculateOneRow(index,comment, emotion, rate):
    lemmatisation_text = re.sub("[!#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~']", '', comment)
    lemmatisation_text = lemmatisation_text.decode('utf8').lower()
    words_list = lemmatisation_dictParser(lemmatisation_text.encode('utf8'))
    [result, number, emotions_list] = detectingEmotionsInText(words_list)
    result_emotion = getEmotionStatusOfText(result)
    return {"text": comment, 'words': len(comment.split()), "lemmatisation": words_list, 'wordsAfterLemmatisation': len(words_list), 'emotionsList': emotions_list, 'wordsFound': number, "resultOfDetecting": result, "detectedEmotion": result_emotion, 'emotion': emotion, 'rate': rate}

def parallelize(data, func):
    print (partitions)
    data_split = np.array_split(data, partitions)
    pool = Pool(cores)
    mapResult = pool.map(func, data_split)
    data = pd.concat(mapResult)
    pool.close()
    pool.join()
    return data

cores = cpu_count()
partitions = cores

ddata = dd.from_pandas(result, npartitions=partitions)

result = ddata.map_partitions(lambda df: df.apply((lambda row: calculateOneRow(*row)), axis=1)).compute(get=get)
result_df = pd.DataFrame()
for o in result:
    result_df = result_df.append(o, ignore_index=True)
    
result_df.to_csv('restaurant_reviews-analisis.csv', encoding='utf-8', index=False) 

In [8]:
%%time
import sys
import re
reload(sys)
sys.setdefaultencoding('utf-8')

result = pd.read_csv('FilmWeb-movies_reviews.csv')
result = result.reset_index()

def calculateOneRow(index, emotion, rate, comment):
    lemmatisation_text = re.sub("[!#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~']", '', comment)
    lemmatisation_text = lemmatisation_text.decode('utf8').lower()
    words_list = lemmatisation_dictParser(lemmatisation_text.encode('utf8'))
    [result, number, emotions_list] = detectingEmotionsInText(words_list)
    result_emotion = getEmotionStatusOfText(result)
    return {"text": comment, 'words': len(comment.split()), "lemmatisation": words_list, 'wordsAfterLemmatisation': len(words_list), 'emotionsList': emotions_list, 'wordsFound': number, "resultOfDetecting": result, "detectedEmotion": result_emotion, 'emotion': emotion, 'rate': rate}

def parallelize(data, func):
    print (partitions)
    data_split = np.array_split(data, partitions)
    pool = Pool(cores)
    mapResult = pool.map(func, data_split)
    data = pd.concat(mapResult)
    pool.close()
    pool.join()
    return data

cores = cpu_count()
partitions = cores

ddata = dd.from_pandas(result, npartitions=partitions)

result = ddata.map_partitions(lambda df: df.apply((lambda row: calculateOneRow(*row)), axis=1)).compute(get=get)
result_df = pd.DataFrame()
for o in result:
    result_df = result_df.append(o, ignore_index=True)
    
result_df.to_csv('FilmWeb-movies_reviews-analysis-dictParser.csv', encoding='utf-8', index=False) 

In [13]:
%%time
import sys
import re
reload(sys)
sys.setdefaultencoding('utf-8')

result = pd.read_csv('mediaKrytyk-commentsRates.csv')
result = result.reset_index()

def calculateOneRow(index,comment, emotion, rate):
    lemmatisation_text = re.sub("[!#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~']", '', comment)
    words_list = lemmatisation_text.split()
    [result, number] = detectingEmotionsInText(words_list)
    result_emotion = getEmotionStatusOfText(result)
    return {"text": comment, 'words': len(comment.split()), "lemmatisation": words_list, 'wordsAfterLemmatisation': len(words_list), 'wordsFound': number, "resultOfDetecting": result, "detectedEmotion": result_emotion, 'emotion': emotion, 'rate': rate}

def parallelize(data, func):
    print (partitions)
    data_split = np.array_split(data, partitions)
    pool = Pool(cores)
    mapResult = pool.map(func, data_split)
    data = pd.concat(mapResult)
    pool.close()
    pool.join()
    return data

cores = cpu_count()
partitions = cores

ddata = dd.from_pandas(result, npartitions=partitions)

result = ddata.map_partitions(lambda df: df.apply((lambda row: calculateOneRow(*row)), axis=1)).compute(get=get)
result_df = pd.DataFrame()
for o in result:
    result_df = result_df.append(o, ignore_index=True)
    
result_df.to_csv('mediaKrytyk-commentsRates-without-lematisation.csv', encoding='utf-8', index=False) 

In [11]:
result = pd.read_csv('mediaKrytyk-commentsRates.csv')

In [12]:
result

,comments,emotion,rate
0,Adam McKay zrealizował film swojego życia. Naj...,positive,9.0
1,"Ten film tak mnie oczarował, że zapomniałem o ...",positive,9.0
2,"Poza nietypowym wykładem, pozostaje także wyśm...",positive,9.0
3,"Nie jest filmem łatwym, w żadnym wypadku. Jest...",positive,9.0
4,"Czar Big Short nie minie do momentu, aż poznam...",positive,9.0
5,"Wręcz genialny film, wykorzystujący inne ujęci...",positive,9.0
6,Niesie duże wartości edukacyjne i przedstawia ...,positive,9.0
7,"Z pewnością nie jest filmem, na który można si...",positive,8.5
8,Świetny film o kruchym i skorumpowanym rynku.,positive,8.5
9,"Filmowa perełka, która gdyby życie i świat był...",positive,8.5


In [25]:
com = pd.read_csv('CSV/FilmWeb-comments-analysis-neutralemotion-results-3.csv')

In [26]:
len(com)

12821

In [36]:
com = com.drop('wordsFound', axis=1)

In [37]:
com

,emotion,rate,text
0,positive,6.0,Strzał w kolano dla wszystkich facetów ...
1,negative,2.0,Śmierdzące gówno w srebrnym celofaniku... ...
2,positive,6.0,Całkowite zaskoczenie O...
3,negative,1.0,"""Boże, zaraz się zrzygam"".. ..."
4,positive,8.0,"Jak ""Zmierzch"" nie wiem..."
5,negative,2.0,Może gdyby ktoś próbował uratować ten film prz...
6,negative,1.0,"Film dla ""ajfonowych"" lemingów z ujemnym IQ. ..."
7,negative,4.0,"""Porno"" dla ubogich (mieszczek) ..."
8,negative,3.0,Słabo Gra aktorska na p...
9,negative,1.0,To najbardziej antyfeministyczna rzecz w dziej...


In [40]:
result = pd.read_csv('opineo-productsReviews.csv')
result = result.reset_index()

In [41]:
result

,index,comment,emotion,rate
0,0,Niestety to żelazko samo nie chce prasować. Je...,negative,3.0
1,1,"Rewelacyjny produkt za dobrą cenę,super wygłąd...",positive,5.0
2,2,"Żelazko użytkuję od niedawna, a oto pierwsze w...",positive,4.0
3,3,"Mam za sobą ponad 50 tys wyprasowanych koszul,...",negative,1.0
4,4,"ciężkie, z dość dużą stopą, bardzo wygodne w u...",positive,5.0
5,5,Pomimo krótkiego czasu użytkowania żelazko spr...,positive,5.0
6,6,"Bardzo dobre żelazko , system przeciw kapaniu ...",positive,4.0
7,7,Po pierwszym prasowaniu - bardzo dobre wrażeni...,positive,4.0
8,8,"żelazko dobrej jakości, ciężkie - co jest chyb...",positive,4.0
9,9,Narazie opini użytkowania nie napisze bo produ...,positive,4.0
